 # pyMBE Tutorial: the Python-based Molecule Builder for ESPResSo.





The objective of pyMBE is to facilitate the set-up of complex molecules into the Molecular Dynamics software [espressomd.org](ESPResSo). pyMBE is specially well-suitedto facilitate setting up constant pH and grand-reaction ensemble simulations in ESPResSo. 

## Table of contents:
* [Introduction](#introduction)
* [How to create particles](#particles)
* [How to create simple polymers](#simple_polymers)
* [How to create complex polymers](#complex_polymers)
* [How to create di-block copolymers](#diblock_copolymers)
* [Practice by creating branched polyampholytes](#branched_polyampholytes)
* [How to create peptides](#peptides)

## Introduction <a class="anchor" id="introduction"></a>

Let us get started by importing pyMBE library and other important libraries for this tutorial, such as ESPResSo.

In [ ]:
# Import pyMBE and  ESPResSo
import pyMBE
import espressomd
from espressomd import interactions

Creating an instance of pyMBE we can make our code shorter and more readable. 

In [ ]:
pmb = pyMBE.pymbe_library(seed=42)

When pyMBE is inicialized, a default system of reduced units is defined. 

* Unit_length = 0.355 nm.
* Unit_charge = 1 elementary charge.
* Temperature = 298.15 K.

The active set of reduced units can be consulted using:

In [ ]:
reduced_unit_set = pmb.get_reduced_units()
print(reduced_unit_set)

This default definition of reduced units can be changed at the convience of the user using the following command:

In [ ]:
pmb.set_reduced_units(unit_length = 0.5*pmb.units.nm,  
                      unit_charge = 5*pmb.units.e)
                      #, temperature=300*pmb.units.K)

NOTE: All input variables will be given to ESPResSo using these reduced units, since it is a convenient choice for the simulation setup. Internally, pyMBE uses Pint library to deal with unit transformations, which in turn should be  used by the user to define its own variables.

Let us now create an instance of the ESPResSo system where we will place our molecules (a square simulation box with length = `box_l`).

In [ ]:
Box_L = 7.5*pmb.units.nm

espresso_system = espressomd.System(box_l = [Box_L.to('reduced_length').magnitude]*3)

print('The side of the simulation box is ', Box_L, '=' ,Box_L.to('reduced_length'))

## How to create particles <a class="anchor" id="particles"></a>

Particles are the smaller objects in the simulation box, which can represent small ions or other small chemical species. In turn, particles can also be used as building blocks for larger molecules or polymers, where they can represent one monomeric unit or part of it. Particle objects are used in pyMBE as input for several of its funcionalities, including to create larger molecules and peptides. The basic properties of a particle are:

In [ ]:
cation_name = 'Na'
pmb.define_particle(name = cation_name, 
                    z = 0, 
                    sigma = 0.35*pmb.units.nm, 
                    epsilon = 1*pmb.units('reduced_energy'))

The properties of the particles are stored in a pandas Dataframe (df). For displaying all the information of the particles one can use the following command:

In [ ]:
pmb.filter_df(pmb_type = 'particle')

One can use pyMBE to create any number of the defined particles into the ESPResSo system.

In [ ]:
N_cations = 20
pmb.create_particle(name = cation_name,
                    number_of_particles = N_cations,
                    espresso_system = espresso_system)

Let's take a look at the new set of particles...

In [ ]:
pmb.filter_df(pmb_type = 'particle')

Let us see the particles that we have created by visualizing our ESPResSo system.

In [ ]:
# Only necessary to produce the pictures used in this tutorial
from PIL import Image
def create_snapshot_of_espresso_system(espresso_system, filename):
    """
    Uses espresso visualizer for creating a snapshot of the current state of the espresso_system

    Args:
        espresso_system(`espressomd.system.System`): Instance of a system object from the espressomd library.
        filename(`str`): Name of the ouput file for the snapshot
    """ 
    from espressomd import visualization
    visualizer = visualization.openGLLive(
            espresso_system, bond_type_radius=[0.3], particle_coloring='type', draw_axis=False, background_color=[1, 1, 1],
    particle_type_colors=[[1.02,0.51,0], # Brown
                        [1,1,1],  # Grey
                        [2.55,0,0], # Red
                        [0,0,2.05],  # Blue
                        [0,0,2.05],  # Blue
                        [2.55,0,0], # Red
                        [2.05,1.02,0]]) # Orange
    visualizer.screenshot(filename)
    return
picture_name = 'cation_system.png'
create_snapshot_of_espresso_system(espresso_system = espresso_system, 
                               filename = picture_name)
img = Image.open(picture_name)
img.show()

To delete an pyMBE object from the system we can use the following command:

In [ ]:
pmb.destroy_pmb_object_in_system(name = cation_name, 
                                 espresso_system = espresso_system)

Now the df should be empty.

In [ ]:
pmb.filter_df(pmb_type = 'particle')


## How to create simple polymers <a class="anchor" id="simple_polymers"></a>

pyMBE can be used to easily construct coarse-grained models of simple polymers. Let us consider a coarse grained model for polydehydroalanaline (PDha) (figure below) in which its monomeric unit can be represented by three beads, as depicted in the schematics below: a backbone bead (grey), a bead for the carboxylic acid group (red) and a bead for the amino group (blue).

<img src="../figs/PDha.png" width=150 height=150 />


To set up such polymer with pyMBE first one has to define the different particles in the monomer.

In [ ]:
PDha_backbone_bead = 'BB-PDha'
PDha_carboxyl_bead = 'COOH-PDha'
PDha_amine_bead = 'NH3-PDha'

pmb.define_particle(name = PDha_backbone_bead, 
                    z = 0, 
                    sigma = 0.4*pmb.units.nm,
                    epsilon = 1*pmb.units('reduced_energy'))

pmb.define_particle(name = PDha_carboxyl_bead, 
                    z = 0, 
                    sigma = 0.5*pmb.units.nm, 
                    epsilon = 1*pmb.units('reduced_energy'))

pmb.define_particle(name = PDha_amine_bead, 
                    z = 0, 
                    sigma = 0.3*pmb.units.nm, 
                    epsilon = 1*pmb.units('reduced_energy'))


Then, one defines the structure of the residue of the polymer. A residue is composed by a `central_bead` where one or various `side_chains` are attached. Each side chain can contain one particle or other residues. 

In [ ]:
PDha_residue = 'PDha_mon'

pmb.define_residue(name = PDha_residue, 
                   central_bead = PDha_backbone_bead,
                   side_chains = [PDha_carboxyl_bead, PDha_amine_bead])


Once done, one has to define a bond for each different type of bond in the polymer. For simplicity, in this tutorial we assume that all bonds are equal and we set-up all bonds using a harmonic potential with the following arbitrary parameters.

In [ ]:
bond_type = 'harmonic'
generic_bond_lenght=0.4 * pmb.units.nm
generic_harmonic_constant = 400 * pmb.units('reduced_energy / reduced_length**2')

harmonic_bond = {'r_0'    : generic_bond_lenght,
                 'k'      : generic_harmonic_constant
                }

pmb.define_bond(bond_type = bond_type,
                bond_parameters = harmonic_bond,
                particle_pairs = [[PDha_backbone_bead, PDha_backbone_bead],
                                  [PDha_backbone_bead, PDha_carboxyl_bead],
                                  [PDha_backbone_bead, PDha_amine_bead]])

pmb.add_bonds_to_espresso(espresso_system = espresso_system)


NOTE: Currently, only harmonic and FENE bonds are supported.

Finally, one can use the residues to define the polymer sequence given by the argument `residue_list`. One needs to add one residue in `residue_list` per each residue in the polymer chain. For instance a decamer should be created as follows:

In [ ]:
PDha_polymer = 'PDha'
N_monomers = 10

pmb.define_molecule(name = PDha_polymer,
                    residue_list = [PDha_residue]*N_monomers)

After defining the polymer, we are ready to create one PdHa polymer in the center of the simulation box.

In [ ]:
N_polymers = 1

pmb.create_molecule(name = PDha_polymer, 
                    number_of_molecules = N_polymers,
                    espresso_system = espresso_system, 
                    position = [[Box_L.to('reduced_length').magnitude/2]*3]) 

We can always track our particles...

In [ ]:
pmb.filter_df(pmb_type = 'particle')

Now, let us see what we have created...

In [ ]:
picture_name = 'PDha_system.png'
create_snapshot_of_espresso_system(espresso_system = espresso_system, 
                              filename = picture_name)
img = Image.open(picture_name)
img.show()

Delete the particles and check that our df is empty.

In [ ]:
pmb.destroy_pmb_object_in_system(name = PDha_polymer, 
                                 espresso_system = espresso_system)
pmb.filter_df(pmb_type = 'particle')

## How to create complex polymers <a class="anchor" id="complex_polymers"></a>

pyMBE can also be used to setup models that requiere more complex side chains, i.e. with more than one bead per side chain. One example of these complex molecules is the poly(N,N-diallylglutamate) (PDAGA), whose structure is depicted in the figure below. Following the logic of the previous example, one would construct PDAGA with pyMBE by defining a `residue` with a `central_bead` for the polymer backbone (grey) and a `side_chain` attached to it. In this case, the group in the side chain of the PDAGA monomer has a complex structure. This group can be coarse-grained by defining another `residue`  composed by a new `central_bead` which represents the cyclic amine group (blue) and two `side_chains` ($\alpha$ and $\beta$ carboxyl) attached to it (red and orange).

<img src="../figs/PDAGA.png" width=150 height=150 />

One can start by defining each different bead of the PDAGA.

In [ ]:
PDAGA_backbone_bead = 'BB-PDAGA'
PDAGA_cyclic_amine_bead = 'NH3-PDAGA'
PDAGA_alpha_carboxyl_bead = 'aCOOH-PDAGA'
PDAGA_beta_carboxyl_bead = 'bCOOH-PDAGA'

pmb.define_particle(name = PDAGA_backbone_bead, 
                    z = 0,
                    sigma = 0.4*pmb.units.nm, 
                    epsilon = 1*pmb.units('reduced_energy'))

pmb.define_particle(name = PDAGA_cyclic_amine_bead, 
                    z = 0, 
                    sigma = 0.3*pmb.units.nm, 
                    epsilon = 1*pmb.units('reduced_energy'))

pmb.define_particle(name = PDAGA_alpha_carboxyl_bead, 
                    z = 0, 
                    sigma = 0.2*pmb.units.nm, 
                    epsilon = 1*pmb.units('reduced_energy'))

pmb.define_particle(name = PDAGA_beta_carboxyl_bead, 
                    z = 0, 
                    sigma = 0.4*pmb.units.nm, 
                    epsilon = 1*pmb.units('reduced_energy'))

The next step is to define the two different residues: 
1. The side chain: two carboxyl beads attached to the cyclic amine bead.

In [ ]:
PDAGA_side_chain_residue = 'PDAGA_side_chain_residue'

pmb.define_residue (name = PDAGA_side_chain_residue,
                    central_bead = PDAGA_cyclic_amine_bead,
                    side_chains = [PDAGA_alpha_carboxyl_bead, PDAGA_beta_carboxyl_bead])

2. Each monomeric unit of the PDAGA: the side chain defined above attached to the backbone.

In [ ]:
PDAGA_monomer_residue = 'PDAGA_monomer_residue'
pmb.define_residue( name = PDAGA_monomer_residue,
                    central_bead = PDAGA_backbone_bead,
                    side_chains = [PDAGA_side_chain_residue])

Then, we need to set the bonds between the particles in a similar way as for the case of the simple polymer.

In [ ]:
bond_type = 'harmonic'
generic_bond_lenght=0.4 * pmb.units.nm
generic_harmonic_constant = 400 * pmb.units('reduced_energy / reduced_length**2')

harmonic_bond = {'r_0'    : generic_bond_lenght,
                 'k'      : generic_harmonic_constant,
                 }

pmb.define_bond(bond_type = bond_type,
                bond_parameters = harmonic_bond,
                particle_pairs = [[PDAGA_backbone_bead, PDAGA_backbone_bead],
                                  [PDAGA_backbone_bead, PDAGA_cyclic_amine_bead],
                                  [PDAGA_alpha_carboxyl_bead, PDAGA_cyclic_amine_bead],
                                  [PDAGA_beta_carboxyl_bead, PDAGA_cyclic_amine_bead]])

pmb.add_bonds_to_espresso(espresso_system = espresso_system)

Now, let us define an octamer of PDAGA.

In [ ]:
PDAGA_polymer = 'PDAGA'
N_monomers = 8

pmb.define_molecule(name = PDAGA_polymer,
                    residue_list = [PDAGA_monomer_residue]*N_monomers)

Finally, we are able to create a PDAGA polymer into the ESPResSo system.

In [ ]:
N_polymers = 1

pmb.create_molecule(name = PDAGA_polymer,
                    number_of_molecules= N_polymers,
                    espresso_system = espresso_system,
                    position = [[Box_L.to('reduced_length').magnitude/2]*3])

Now, let us see our PDAGA molecule.

In [ ]:
picture_name = 'PDAGA_system.png'
create_snapshot_of_espresso_system(espresso_system = espresso_system, 
                               filename = picture_name)
img = Image.open(picture_name)
img.show()


Delete the particles and check that our df is empty.

In [ ]:
pmb.destroy_pmb_object_in_system(name = PDAGA_polymer, 
                                 espresso_system = espresso_system)
pmb.filter_df(pmb_type = 'particle')

## How to create di-block copolymers <a class="anchor" id="diblock_copolymers"></a>

In turn, the residues previously defined to build the PDAGA and PDha molecules can be used to build more complex polymers such as a di-block PDha-PDAGA copolymer, as shown in the picture below

<img src="../figs/PDAGA_PDha_diblock_copolymer.png" width=250 height=250 />

Defining each different bead of the PDha and PDAGA.

In [ ]:
PDha_backbone_bead = 'BB-PDha'
PDha_carboxyl_bead = 'COOH-PDha'
PDha_amine_bead = 'NH3-PDha'

pmb.define_particle(name = PDha_backbone_bead, 
                    z = 0,
                    sigma = 0.4*pmb.units.nm,
                    epsilon = 1*pmb.units('reduced_energy'))

pmb.define_particle(name = PDha_carboxyl_bead,
                    z = 0,
                    sigma = 0.5*pmb.units.nm,
                    epsilon = 1*pmb.units('reduced_energy'))

pmb.define_particle(name = PDha_amine_bead,
                    z = 0,
                    sigma = 0.3*pmb.units.nm,
                    epsilon = 1*pmb.units('reduced_energy'))

PDAGA_backbone_bead = 'BB-PDAGA'
PDAGA_cyclic_amine_bead = 'NH3-PDAGA'
PDAGA_alpha_carboxyl_bead = 'aCOOH-PDAGA'
PDAGA_beta_carboxyl_bead = 'bCOOH-PDAGA'


pmb.define_particle(name = PDAGA_backbone_bead,
                    z = 0,
                    sigma = 0.4*pmb.units.nm, 
                    epsilon = 1*pmb.units('reduced_energy'))

pmb.define_particle(name = PDAGA_cyclic_amine_bead,
                    z = 0, 
                    sigma = 0.3*pmb.units.nm,
                    epsilon = 1*pmb.units('reduced_energy'))

pmb.define_particle(name = PDAGA_alpha_carboxyl_bead,
                    z = 0,
                    sigma = 0.2*pmb.units.nm,
                    epsilon = 1*pmb.units('reduced_energy'))

pmb.define_particle(name = PDAGA_beta_carboxyl_bead, 
                    z = 0,
                    sigma = 0.4*pmb.units.nm,
                    epsilon = 1*pmb.units('reduced_energy'))

Defining the different residues for the PDha and PDAGA.

In [ ]:
bond_type = 'harmonic'
generic_bond_lenght=0.4 * pmb.units.nm
generic_harmonic_constant = 400 * pmb.units('reduced_energy / reduced_length**2')

harmonic_bond = {'r_0'    : generic_bond_lenght,
                 'k'      : generic_harmonic_constant,
                 }

######################################################################
################################# PDha ###############################
######################################################################

PDha_residue = 'PDha_residue'
pmb.define_residue(name = PDha_residue, 
                   central_bead =  PDha_backbone_bead ,
                   side_chains = [PDha_carboxyl_bead,PDha_amine_bead])


pmb.define_bond(bond_type = bond_type,
                bond_parameters = harmonic_bond,
                particle_pairs = [[PDha_backbone_bead, PDha_backbone_bead],
                                  [PDha_backbone_bead, PDha_carboxyl_bead],
                                  [PDha_backbone_bead, PDha_amine_bead]])

######################################################################
################################ PDAGA ###############################
######################################################################

PDAGA_monomer_residue = 'PDAGA_monomer_residue'
pmb.define_residue( name = PDAGA_monomer_residue,
                    central_bead = PDAGA_backbone_bead,
                    side_chains = [PDAGA_side_chain_residue])

PDAGA_side_chain_residue = 'PDAGA_side_chain_residue'
pmb.define_residue (name = PDAGA_side_chain_residue,
                    central_bead = PDAGA_cyclic_amine_bead,
                    side_chains = [PDAGA_alpha_carboxyl_bead,PDAGA_beta_carboxyl_bead])

pmb.define_bond(bond_type = bond_type,
                bond_parameters = harmonic_bond,
                particle_pairs = [[PDAGA_backbone_bead, PDAGA_backbone_bead],
                                  [PDAGA_backbone_bead, PDAGA_cyclic_amine_bead],
                                  [PDAGA_alpha_carboxyl_bead, PDAGA_cyclic_amine_bead],
                                  [PDAGA_beta_carboxyl_bead, PDAGA_cyclic_amine_bead]])

######################################################################
############################# PDha - PDAGA ###########################
######################################################################

pmb.define_bond(bond_type = bond_type,
                bond_parameters = harmonic_bond,
                particle_pairs = [[PDha_backbone_bead, PDAGA_backbone_bead]])

pmb.add_bonds_to_espresso(espresso_system = espresso_system)

Defining the di-block polymer molecule

In [ ]:
N_monomers_PDha = 4
N_monomers_PDAGA = 4
diblock_polymer = 'diblock'

pmb.define_molecule(name = diblock_polymer,
                    residue_list = [PDha_residue]*N_monomers_PDha+[PDAGA_monomer_residue]*N_monomers_PDAGA)

Creating the di-block polymer into the ESPResSo system

In [ ]:
N_polymers = 1

pmb.create_molecule(name = diblock_polymer,
                    number_of_molecules= N_polymers,
                    espresso_system = espresso_system,
                    position = [[Box_L.to('reduced_length').magnitude/2]*3]) 

Now, let us see our di-block PDha-PDAGA molecule.

In [ ]:
picture_name = 'diblock_system.png'
create_snapshot_of_espresso_system(espresso_system = espresso_system, 
                               filename = picture_name)
img = Image.open(picture_name)
img.show()

Delete the particles and check that our df is empty.

In [ ]:
pmb.destroy_pmb_object_in_system(name = diblock_polymer, 
                     espresso_system = espresso_system)
pmb.filter_df(pmb_type = 'particle')

## Practice by creating a custom polyampholyte chain <a class="anchor" id="branched_polyampholytes"></a>

Polyampholytes are polymers containing both acidic and basic groups on the same molecule, one example of a branched polyampholyte is depicted in the figure below.

<img src="../figs/branched_polyampholyte.png" width=350 height=350 />

We will create the polyampholyte chain in the figure, starting by defining two different residues, 'Res_1' and 'Res_2'. The polyampholyte chain is then defined by following residue_list:

residue_list = 2*["Res_1"] + ["Res_2"] + 2*["Res_1"] + 2*["Res_2"]

### Tasks to do:

1. Define each different bead in the residues using "pmb.define_particle". There are 3 different particles, an inert particle, an acidic particle with pKa = 4, and a basic particle with pKa = 9.
2. Define residues using "pmf.define_residue". "Res_1" consists of an inert particle as central bead and acidic and basic particles as side chain. "Res_2" consists of an inert particle as central bead and "Res_1" as side chain.  
3. Define the branched polyampholyte chain using "pmb.define_molecule" with the above "residue_list." 
4. Create the branched polyampholyte into the ESPResSo system.
5. Visualize your creation.
6. Delete the molecule and check that your df is empty.

#### 1. Define each different bead of Res_1 and Res_2.

#### 2. Define the residues Res_1 and Res_2.

#### 3. Define the diblock polyampholyte molecule. 

#### 4. Create the diblock polyampholyte chain into the ESPResSo system.

#### 5. Visualize your creation.

#### 6. Delete the molecule and check that your df is empty.

Refer to the sample script "branched_polyampholyte.py" in the samples folder for a complete solution of this exercise.

## How to create peptides <a class="anchor" id="peptides"></a>

pyMBE includes built-on functions to facilitate the setting up of coarse-grained models for peptides from their aminoacid sequence. Currently, there are two different coarse-grained models implemented: 

* `1beadAA`, where the aminoacid is represented by one single bead.
* `2beadAA`, where the aminoacid is represented by two beads (backbone and side-chain). 

We provide reference parameters in the folder (`parameters`) which can be loaded into pyMBE. The peptide sequence should be provided as a str composed either by the list of the one letter code or the list of the three letter code of the corresponding aminoacids. For example, the two possible ways to provide the peptide Cysteine$_3$ - Glutamic acid$_2$ - Histidine$_4$ - Valine are:

* one letter code: 'CCCEEHHHHV'
* three letter code: 'CYS-CYS-CYS-GLU-GLU-HIS-HIS-HIS-HIS-VAL'

Let's set up the peptide Lysine$_5$ - Glutamic acid$_5$ using a two beads coarse-grained model.

In [ ]:
N_peptide = 1
sequence = "KKKKKEEEEE"
model = '2beadAA'

We can use the peptide parametrization reported by Lunkad et al., which is provided in the reference folder. After loading the parameters we should add the bonds to the ESPResSo system. 

In [ ]:
path_to_interactions=pmb.root / "parameters" / "peptides" / "Lunkad2021.json"
path_to_pka=pmb.root / "parameters" / "pka_sets" / "Hass2015.json"
pmb.load_interaction_parameters(filename=path_to_interactions) 
pmb.load_pka_set (path_to_pka)
pmb.add_bonds_to_espresso(espresso_system = espresso_system)

In [ ]:
print(pmb.df)

Now, we can define our peptide and create it into the ESPResSo system. 

In [ ]:
pmb.define_peptide(name = sequence, 
                   sequence = sequence, 
                   model = model)

pmb.create_molecule(name = sequence,
                    number_of_molecules=  N_peptide,
                    espresso_system = espresso_system,
                    position = [[Box_L.to('reduced_length').magnitude/2]*3])

Let us visualize our peptide.

In [ ]:
picture_name = 'peptide.png'
create_snapshot_of_espresso_system(espresso_system = espresso_system, 
                               filename = picture_name)
img = Image.open(picture_name)
img.show()

Delete the particles and check that our df is empty.

In [ ]:
pmb.destroy_pmb_object_in_system(name = sequence, 
                                 espresso_system = espresso_system)

Finally, the setup using the three letter code. pyMBE automatically detects and transforms into the one-letter code using its own protein sequence parser.

In [ ]:
sequence = 'LYS-LYS-LYS-LYS-LYS-GLU-GLU-GLU-GLU-GLU'

pmb.define_peptide(name = sequence, 
                   sequence = sequence, 
                   model = model)

print('one letter code', pmb.protein_sequence_parser(sequence=sequence))
print('defined peptide sequence ', sequence)

### References

Lunkad, R. et al.  Molecular Systems Design & Engineering (2021), 6(2), 122-131.